In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB:
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [2]:
# csv to df
path = os.path.abspath('politifact_balanced_data.csv')
df = pd.read_csv(path, index_col=0)

# specify rows of importance
df = df[["veracity", "statement"]]

# See sample of data
df.head(5)

,veracity,statement
3,0,"""Tim Kaine doesn’t want a border at all. He wa..."
5,0,"""The deficit ... is coming down, and it’s comi..."
20,1,"""Migrant mother and ‘crying girl’ on Time cove..."
23,0,"""Fact: Over 90,000 kids were detained under Ob..."
24,0,"""$1 billion—that’s how much Bruce Rauner has w..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2165 entries, 3 to 11183
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   veracity   2165 non-null   int64 
 1   statement  2165 non-null   object
dtypes: int64(1), object(1)
memory usage: 50.7+ KB


In [4]:
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
df_validation = df.drop(train.index)
validation=df_validation.sample(frac=0.6,random_state=200)
test = df_validation.drop(validation.index)

In [5]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1732 entries, 698 to 4956
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   veracity   1732 non-null   int64 
 1   statement  1732 non-null   object
dtypes: int64(1), object(1)
memory usage: 40.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 70 to 11183
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   veracity   173 non-null    int64 
 1   statement  173 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.1+ KB


# Transforming the statements into tensors

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

train_text = train['statement'].to_numpy()
tok = Tokenizer(oov_token='<unk>')
tok.fit_on_texts(train_text)
tok.word_index['<pad>'] = 0
tok.index_word[0] = '<pad>'

train_seqs = tok.texts_to_sequences(train_text)
train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

train_labels = train['veracity'].to_numpy().flatten()


# CONVERT TO TF DATASETS

train_ds = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))

BATCH_SIZE = 32
BUFFER_SIZE = 7
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [7]:
embed_size = 128
num_oov_buckets = 1000
vocab_size=10000
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_ds, epochs=5)

Epoch 1/5
55/55 [==============================] - 10s 76ms/step - loss: 0.6836 - accuracy: 0.5508
Epoch 2/5
55/55 [==============================] - 4s 76ms/step - loss: 0.4578 - accuracy: 0.7846
Epoch 3/5
55/55 [==============================] - 4s 75ms/step - loss: 0.1819 - accuracy: 0.9342
Epoch 4/5
55/55 [==============================] - 4s 74ms/step - loss: 0.0526 - accuracy: 0.9815
Epoch 5/5
55/55 [==============================] - 4s 74ms/step - loss: 0.0310 - accuracy: 0.9902


In [8]:
history.history

{'loss': [0.6836201548576355,
  0.4578196406364441,
  0.18188466131687164,
  0.052647653967142105,
  0.031013986095786095],
 'accuracy': [0.550808310508728,
  0.7846420407295227,
  0.9341801404953003,
  0.9815242290496826,
  0.9901847839355469]}

In [9]:
test_text = test['statement'].to_numpy()
test_seqs = tok.texts_to_sequences(test_text)
test_seqs = tf.keras.preprocessing.sequence.pad_sequences(test_seqs, padding='post')

test_labels = test['veracity'].to_numpy().flatten()

results = model.evaluate(test_seqs, test_labels, batch_size=32)

6/6 [==============================] - 2s 24ms/step - loss: 1.9571 - accuracy: 0.5723


# Avoid Overfitting Through Regularization

In [19]:
train=df.sample(frac=0.65,random_state=200) #random state is a seed value
df_validation = df.drop(train.index)
validation=df_validation.sample(frac=0.6,random_state=200)
test = df_validation.drop(validation.index)

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer

train_text = train['statement'].to_numpy()
tok = Tokenizer(oov_token='<unk>')
tok.fit_on_texts(train_text)
tok.word_index['<pad>'] = 0
tok.index_word[0] = '<pad>'

train_seqs = tok.texts_to_sequences(train_text)
train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

train_labels = train['veracity'].to_numpy().flatten()

valid_text = validation['statement'].to_numpy()
valid_seqs = tok.texts_to_sequences(valid_text)
valid_seqs = tf.keras.preprocessing.sequence.pad_sequences(valid_seqs, padding='post')

valid_labels = validation['veracity'].to_numpy().flatten()



# CONVERT TO TF DATASETS

train_ds = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_seqs,valid_labels))

BATCH_SIZE = 32
BUFFER_SIZE = 7
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_ds = valid_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [24]:
embed_size = 128
num_oov_buckets = 1000
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(
    train_ds,
    epochs=5,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/5
7/7 [==============================] - 8s 387ms/step - loss: 0.6941 - accuracy: 0.4554 - val_loss: 0.6924 - val_accuracy: 0.5000
Epoch 2/5
7/7 [==============================] - 1s 85ms/step - loss: 0.6927 - accuracy: 0.5134 - val_loss: 0.6894 - val_accuracy: 0.5625
Epoch 3/5
7/7 [==============================] - 1s 82ms/step - loss: 0.6859 - accuracy: 0.5491 - val_loss: 0.6872 - val_accuracy: 0.5625
Epoch 4/5
7/7 [==============================] - 1s 81ms/step - loss: 0.6969 - accuracy: 0.4821 - val_loss: 0.6873 - val_accuracy: 0.4688
Epoch 5/5
7/7 [==============================] - 1s 83ms/step - loss: 0.6806 - accuracy: 0.5893 - val_loss: 0.6801 - val_accuracy: 0.4688


In [28]:
from functools import partial



model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.GRU(128, kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model.fit(
    train_ds,
    epochs=5,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/5
7/7 [==============================] - 8s 395ms/step - loss: 4.2884 - accuracy: 0.4777 - val_loss: 3.9875 - val_accuracy: 0.5312
Epoch 2/5
7/7 [==============================] - 1s 88ms/step - loss: 3.7639 - accuracy: 0.5179 - val_loss: 3.4686 - val_accuracy: 0.5312
Epoch 3/5
7/7 [==============================] - 1s 88ms/step - loss: 3.2544 - accuracy: 0.5536 - val_loss: 2.9857 - val_accuracy: 0.5312
Epoch 4/5
7/7 [==============================] - 1s 89ms/step - loss: 2.8071 - accuracy: 0.4643 - val_loss: 2.5657 - val_accuracy: 0.5625
Epoch 5/5
7/7 [==============================] - 1s 88ms/step - loss: 2.4063 - accuracy: 0.5357 - val_loss: 2.2075 - val_accuracy: 0.5312


In [29]:
history1 = model.fit( train_ds,
    epochs=5)

Epoch 1/5
44/44 [==============================] - 4s 86ms/step - loss: 1.5065 - accuracy: 0.5963
Epoch 2/5
44/44 [==============================] - 4s 82ms/step - loss: 0.7355 - accuracy: 0.7562
Epoch 3/5
44/44 [==============================] - 4s 83ms/step - loss: 0.3813 - accuracy: 0.8678
Epoch 4/5
44/44 [==============================] - 4s 82ms/step - loss: 0.2279 - accuracy: 0.9318
Epoch 5/5
44/44 [==============================] - 4s 83ms/step - loss: 0.1177 - accuracy: 0.9765


In [30]:
history1.history

{'loss': [1.5064618587493896,
  0.7355286478996277,
  0.38134974241256714,
  0.22788342833518982,
  0.1176784411072731],
 'accuracy': [0.596304178237915,
  0.7562189102172852,
  0.8678038120269775,
  0.9317697286605835,
  0.9765458703041077]}

In [31]:
test_text = test['statement'].to_numpy()
test_seqs = tok.texts_to_sequences(test_text)
test_seqs = tf.keras.preprocessing.sequence.pad_sequences(test_seqs, padding='post')

test_labels = test['veracity'].to_numpy().flatten()

In [34]:
results = model.evaluate(test_seqs, test_labels, batch_size=32)

10/10 [==============================] - 0s 36ms/step - loss: 1.9703 - accuracy: 0.5413


# Adding l2 regularization acctually slightly improved the performance of the model

# Using Dropout 

In [45]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(
    train_ds,
    epochs=7,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/7
7/7 [==============================] - 12s 654ms/step - loss: 0.6947 - accuracy: 0.4732 - val_loss: 0.6933 - val_accuracy: 0.5312
Epoch 2/7
7/7 [==============================] - 2s 221ms/step - loss: 0.6908 - accuracy: 0.5312 - val_loss: 0.6902 - val_accuracy: 0.5312
Epoch 3/7
7/7 [==============================] - 2s 217ms/step - loss: 0.6869 - accuracy: 0.5491 - val_loss: 0.6883 - val_accuracy: 0.5312
Epoch 4/7
7/7 [==============================] - 2s 220ms/step - loss: 0.7004 - accuracy: 0.4643 - val_loss: 0.6869 - val_accuracy: 0.6562
Epoch 5/7
7/7 [==============================] - 2s 222ms/step - loss: 0.6880 - accuracy: 0.5179 - val_loss: 0.6818 - val_accuracy: 0.5938
Epoch 6/7
7/7 [==============================] - 2s 222ms/step - loss: 0.6852 - accuracy: 0.5670 - val_loss: 0.6631 - val_accuracy: 0.6875
Epoch 7/7
7/7 [==============================] - 0s 48ms/step - loss: 0.6906 - accuracy: 0.5079 - val_loss: 0.6637 - val_accuracy: 0.6875


# Using dropout er attained a respectable 68% accuracy on the validation data

In [46]:
results = model.evaluate(test_seqs, test_labels, batch_size=32)

10/10 [==============================] - 4s 85ms/step - loss: 0.6862 - accuracy: 0.5413


# Tuning parameters in order to obtain better results

In [49]:
model1 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.Dropout(rate=0.4),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.3),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.3),
    keras.layers.GRU(70),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Dense(1, activation="sigmoid")
])
model1.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model1.fit(
    train_ds,
    epochs=7,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/7
7/7 [==============================] - 12s 674ms/step - loss: 0.6947 - accuracy: 0.4420 - val_loss: 0.6925 - val_accuracy: 0.5312
Epoch 2/7
7/7 [==============================] - 2s 216ms/step - loss: 0.6925 - accuracy: 0.5223 - val_loss: 0.6921 - val_accuracy: 0.5312
Epoch 3/7
7/7 [==============================] - 2s 216ms/step - loss: 0.6908 - accuracy: 0.5268 - val_loss: 0.6923 - val_accuracy: 0.5312
Epoch 4/7
7/7 [==============================] - 2s 218ms/step - loss: 0.6972 - accuracy: 0.4509 - val_loss: 0.6945 - val_accuracy: 0.4688
Epoch 5/7
7/7 [==============================] - 2s 217ms/step - loss: 0.6918 - accuracy: 0.5312 - val_loss: 0.6888 - val_accuracy: 0.4688
Epoch 6/7
7/7 [==============================] - 1s 209ms/step - loss: 0.6948 - accuracy: 0.5223 - val_loss: 0.6887 - val_accuracy: 0.5938
Epoch 7/7
7/7 [==============================] - 1s 52ms/step - loss: 0.6836 - accuracy: 0.6190 - val_loss: 0.6940 - val_accuracy: 0.4375


In [52]:
model1 = keras.models.Sequential([
    keras.layers.Embedding( + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.Dropout(rate=0.4),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.3),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.3),
    keras.layers.GRU(128),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Dense(1, activation="sigmoid")
])
model1.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model1.fit(
    train_ds,
    epochs=6,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/6
7/7 [==============================] - 13s 663ms/step - loss: 0.6922 - accuracy: 0.5446 - val_loss: 0.6914 - val_accuracy: 0.5312
Epoch 2/6
7/7 [==============================] - 2s 218ms/step - loss: 0.6929 - accuracy: 0.5312 - val_loss: 0.6908 - val_accuracy: 0.5312
Epoch 3/6
7/7 [==============================] - 1s 211ms/step - loss: 0.6904 - accuracy: 0.4955 - val_loss: 0.6911 - val_accuracy: 0.5312
Epoch 4/6
7/7 [==============================] - 1s 213ms/step - loss: 0.6989 - accuracy: 0.4464 - val_loss: 0.6947 - val_accuracy: 0.4062
Epoch 5/6
7/7 [==============================] - 1s 210ms/step - loss: 0.6895 - accuracy: 0.5134 - val_loss: 0.6870 - val_accuracy: 0.5938
Epoch 6/6
7/7 [==============================] - 2s 245ms/step - loss: 0.7000 - accuracy: 0.5491 - val_loss: 0.6879 - val_accuracy: 0.5938


#Try decreasing the vocab size

In [54]:
model2 = keras.models.Sequential([
    keras.layers.Embedding(6000 + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.Dropout(rate=0.4),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model2.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model2.fit(
    train_ds,
    epochs=6,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/6
7/7 [==============================] - 13s 682ms/step - loss: 0.6944 - accuracy: 0.4732 - val_loss: 0.6906 - val_accuracy: 0.5312
Epoch 2/6
7/7 [==============================] - 2s 220ms/step - loss: 0.6921 - accuracy: 0.5179 - val_loss: 0.6887 - val_accuracy: 0.5312
Epoch 3/6
7/7 [==============================] - 2s 215ms/step - loss: 0.6879 - accuracy: 0.5268 - val_loss: 0.6878 - val_accuracy: 0.5312
Epoch 4/6
7/7 [==============================] - 2s 220ms/step - loss: 0.6976 - accuracy: 0.4911 - val_loss: 0.6918 - val_accuracy: 0.5000
Epoch 5/6
7/7 [==============================] - 1s 210ms/step - loss: 0.6961 - accuracy: 0.5223 - val_loss: 0.6833 - val_accuracy: 0.5625
Epoch 6/6
7/7 [==============================] - 1s 214ms/step - loss: 0.6901 - accuracy: 0.5446 - val_loss: 0.6805 - val_accuracy: 0.5938


In [56]:
model2 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.Dropout(rate=0.4),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model2.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model2.fit(
    train_ds,
    epochs=6,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/6
7/7 [==============================] - 17s 1s/step - loss: 0.6933 - accuracy: 0.5268 - val_loss: 0.6908 - val_accuracy: 0.5312
Epoch 2/6
7/7 [==============================] - 2s 300ms/step - loss: 0.6936 - accuracy: 0.5134 - val_loss: 0.6894 - val_accuracy: 0.5312
Epoch 3/6
7/7 [==============================] - 2s 292ms/step - loss: 0.6885 - accuracy: 0.5536 - val_loss: 0.6891 - val_accuracy: 0.5312
Epoch 4/6
7/7 [==============================] - 2s 305ms/step - loss: 0.6996 - accuracy: 0.4464 - val_loss: 0.6926 - val_accuracy: 0.4688
Epoch 5/6
7/7 [==============================] - 2s 302ms/step - loss: 0.6919 - accuracy: 0.5000 - val_loss: 0.6796 - val_accuracy: 0.6875
Epoch 6/6
7/7 [==============================] - 2s 292ms/step - loss: 0.6963 - accuracy: 0.5580 - val_loss: 0.6779 - val_accuracy: 0.6875


# Determine if increasing the number of layers improves performance overall

In [58]:
model3 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.Dropout(rate=0.4),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model3.fit(
    train_ds,
    epochs=6,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/6
7/7 [==============================] - 18s 975ms/step - loss: 0.6941 - accuracy: 0.4955 - val_loss: 0.6914 - val_accuracy: 0.5312
Epoch 2/6
7/7 [==============================] - 3s 393ms/step - loss: 0.6937 - accuracy: 0.5179 - val_loss: 0.6904 - val_accuracy: 0.5312
Epoch 3/6
7/7 [==============================] - 3s 391ms/step - loss: 0.6890 - accuracy: 0.5536 - val_loss: 0.6897 - val_accuracy: 0.5312
Epoch 4/6
7/7 [==============================] - 3s 396ms/step - loss: 0.6975 - accuracy: 0.4598 - val_loss: 0.6910 - val_accuracy: 0.5312
Epoch 5/6
7/7 [==============================] - 3s 381ms/step - loss: 0.6917 - accuracy: 0.5357 - val_loss: 0.6874 - val_accuracy: 0.5625
Epoch 6/6
7/7 [==============================] - 3s 384ms/step - loss: 0.6869 - accuracy: 0.5625 - val_loss: 0.6767 - val_accuracy: 0.6875


In [60]:
model4 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model4.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model4.fit(
    train_ds,
    epochs=6,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/6
7/7 [==============================] - 23s 1s/step - loss: 0.6949 - accuracy: 0.4554 - val_loss: 0.6916 - val_accuracy: 0.5312
Epoch 2/6
7/7 [==============================] - 4s 565ms/step - loss: 0.6926 - accuracy: 0.5312 - val_loss: 0.6908 - val_accuracy: 0.5312
Epoch 3/6
7/7 [==============================] - 4s 562ms/step - loss: 0.6868 - accuracy: 0.5714 - val_loss: 0.6928 - val_accuracy: 0.5312
Epoch 4/6
7/7 [==============================] - 4s 565ms/step - loss: 0.6931 - accuracy: 0.5134 - val_loss: 0.7029 - val_accuracy: 0.4688
Epoch 5/6
7/7 [==============================] - 4s 569ms/step - loss: 0.6943 - accuracy: 0.4955 - val_loss: 0.6890 - val_accuracy: 0.5938
Epoch 6/6
7/7 [==============================] - 4s 563ms/step - loss: 0.6970 - accuracy: 0.5714 - val_loss: 0.6866 - val_accuracy: 0.5312


## Model 2 has the most effective arhitecture so far, so we are going to improve on that 

In [63]:
model21 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.AlphaDropout(rate=0.4),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model21.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model21.fit(
    train_ds,
    epochs=6,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/6
7/7 [==============================] - 20s 1s/step - loss: 0.7993 - accuracy: 0.4821 - val_loss: 0.6913 - val_accuracy: 0.5312
Epoch 2/6
7/7 [==============================] - 4s 525ms/step - loss: 0.7344 - accuracy: 0.5491 - val_loss: 0.6909 - val_accuracy: 0.5312
Epoch 3/6
7/7 [==============================] - 4s 532ms/step - loss: 0.7856 - accuracy: 0.4911 - val_loss: 0.6919 - val_accuracy: 0.5312
Epoch 4/6
7/7 [==============================] - 4s 526ms/step - loss: 0.8291 - accuracy: 0.4732 - val_loss: 0.6915 - val_accuracy: 0.5312
Epoch 5/6
7/7 [==============================] - 4s 546ms/step - loss: 0.7825 - accuracy: 0.4955 - val_loss: 0.6907 - val_accuracy: 0.5312
Epoch 6/6
7/7 [==============================] - 4s 530ms/step - loss: 0.7559 - accuracy: 0.5223 - val_loss: 0.6948 - val_accuracy: 0.4688


# AlphaDropout does not add any improvement over simple Dropout

## adding a more complex optimizer

In [66]:
model22 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.AlphaDropout(rate=0.4),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
optimizer = keras.optimizers.SGD(learning_rate=0.02, momentum=0.8, nesterov=True)
model22.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model22.fit(
    train_ds,
    epochs=6,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/6
7/7 [==============================] - 18s 1s/step - loss: 0.7412 - accuracy: 0.5268 - val_loss: 0.6927 - val_accuracy: 0.5312
Epoch 2/6
7/7 [==============================] - 4s 573ms/step - loss: 0.8445 - accuracy: 0.4598 - val_loss: 0.6915 - val_accuracy: 0.5312
Epoch 3/6
7/7 [==============================] - 4s 561ms/step - loss: 0.8005 - accuracy: 0.4732 - val_loss: 0.6912 - val_accuracy: 0.5312
Epoch 4/6
7/7 [==============================] - 4s 554ms/step - loss: 0.7528 - accuracy: 0.5402 - val_loss: 0.6917 - val_accuracy: 0.5312
Epoch 5/6
7/7 [==============================] - 4s 571ms/step - loss: 0.7065 - accuracy: 0.5625 - val_loss: 0.6913 - val_accuracy: 0.5312
Epoch 6/6
7/7 [==============================] - 4s 577ms/step - loss: 0.7073 - accuracy: 0.5714 - val_loss: 0.6917 - val_accuracy: 0.5312


# Adding Max Norm

In [67]:
from functools import partial

In [70]:
MaxNormGRU = partial(keras.layers.GRU,
                       return_sequences=True,
                       kernel_constraint=keras.constraints.max_norm(1.))

model24 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.Dropout(rate=0.4),
    MaxNormGRU(50),
    keras.layers.Dropout(rate=0.2),
    MaxNormGRU(50),
    keras.layers.Dropout(rate=0.2),
    MaxNormGRU(50),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(50),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])

model24.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model24.fit(
    train_ds,
    epochs=5,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

Epoch 1/6
7/7 [==============================] - 16s 823ms/step - loss: 0.7874 - accuracy: 0.4732 - val_loss: 0.6920 - val_accuracy: 0.5312
Epoch 2/6
7/7 [==============================] - 2s 226ms/step - loss: 0.7606 - accuracy: 0.4866 - val_loss: 0.6907 - val_accuracy: 0.5312
Epoch 3/6
7/7 [==============================] - 2s 228ms/step - loss: 0.7513 - accuracy: 0.5312 - val_loss: 0.6905 - val_accuracy: 0.5312
Epoch 4/6
7/7 [==============================] - 2s 227ms/step - loss: 0.8101 - accuracy: 0.4643 - val_loss: 0.6928 - val_accuracy: 0.4375
Epoch 5/6
7/7 [==============================] - 2s 230ms/step - loss: 0.7499 - accuracy: 0.5089 - val_loss: 0.6868 - val_accuracy: 0.5625
Epoch 6/6
7/7 [==============================] - 2s 230ms/step - loss: 0.7246 - accuracy: 0.5625 - val_loss: 0.6930 - val_accuracy: 0.4688


In [73]:
embed_size = 128
num_oov_buckets = 1000
vocab_size=10000
model30 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model30.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model30.fit(train_ds, epochs=5)

Epoch 1/5
44/44 [==============================] - 18s 279ms/step - loss: 0.6907 - accuracy: 0.5323
Epoch 2/5
44/44 [==============================] - 12s 277ms/step - loss: 0.5238 - accuracy: 0.7520
Epoch 3/5
44/44 [==============================] - 12s 279ms/step - loss: 0.1927 - accuracy: 0.9190
Epoch 4/5
44/44 [==============================] - 12s 275ms/step - loss: 0.0397 - accuracy: 0.9865
Epoch 5/5
44/44 [==============================] - 12s 278ms/step - loss: 0.0080 - accuracy: 0.9979


In [74]:
results = model30.evaluate(test_seqs, test_labels, batch_size=32)

10/10 [==============================] - 3s 89ms/step - loss: 2.4807 - accuracy: 0.5248


In [80]:
results2 = model30.evaluate(valid_seqs, valid_labels, batch_size=10)

46/46 [==============================] - 2s 38ms/step - loss: 2.1155 - accuracy: 0.5824


In [84]:
restults3 = model3.evaluate(test_seqs, test_labels, batch_size=18)

17/17 [==============================] - 2s 109ms/step - loss: 0.6827 - accuracy: 0.5677


In [86]:
restults4 = model2.evaluate(valid_seqs, valid_labels, batch_size=10)

46/46 [==============================] - 3s 69ms/step - loss: 0.6705 - accuracy: 0.5956


In [88]:
embed_size = 128
num_oov_buckets = 1000
vocab_size=10000
model33 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True, kernel_constraint=keras.constraints.max_norm(1.)),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128, return_sequences=True, kernel_constraint=keras.constraints.max_norm(1.)),
    keras.layers.Dropout(rate=0.2),
    keras.layers.GRU(128),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model33.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model33.fit(train_ds, epochs=5)

Epoch 1/5
44/44 [==============================] - 27s 409ms/step - loss: 0.6916 - accuracy: 0.5167
Epoch 2/5
44/44 [==============================] - 18s 414ms/step - loss: 0.5198 - accuracy: 0.7392
Epoch 3/5
44/44 [==============================] - 18s 417ms/step - loss: 0.2429 - accuracy: 0.9055
Epoch 4/5
44/44 [==============================] - 18s 416ms/step - loss: 0.0648 - accuracy: 0.9780
Epoch 5/5
44/44 [==============================] - 18s 419ms/step - loss: 0.0245 - accuracy: 0.9908


In [89]:
restults5 = model33.evaluate(test_seqs, test_labels, batch_size=18)

17/17 [==============================] - 4s 73ms/step - loss: 2.2694 - accuracy: 0.5314


In [90]:
restults3 = model3.evaluate(valid_seqs, valid_labels, batch_size=32)

15/15 [==============================] - 3s 165ms/step - loss: 0.6837 - accuracy: 0.5385
